In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch
import torchaudio
import random

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int
    apply_augmentation: bool = False
    n_fft_choices: List[int] = (400, 800, 1024)
    hop_length_choices: List[int] = (160, 320, 512)

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = []
        labels_list = []
        dec_input_features = []
        
        for feature in features:
            audio = feature["input_features"]
            if self.apply_augmentation:
                # Randomly select n_fft and hop_length for augmentation
                n_fft = random.choice(self.n_fft_choices)
                hop_length = random.choice(self.hop_length_choices)
            else:
                # Use default values if augmentation is not applied
                n_fft = 1024
                hop_length = 512

            # Apply MelSpectrogram transformation with the selected parameters
            mel_spectrogram = torchaudio.transforms.MelSpectrogram(
                sample_rate=16000,  # Sample rate is assumed; update if necessary
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=80
            )(torch.tensor(audio))

            log_mel_spectrogram = torch.log(mel_spectrogram + 1e-9)
            input_features.append({"input_features": log_mel_spectrogram})
            
            label = feature["labels"]
            label_tokens = [self.processor.tokenizer.bos_token_id] + self.processor.tokenizer.encode(label) + [self.processor.tokenizer.eos_token_id]
            dec_input_feature = label_tokens[:-1]
            label = label_tokens[1:]
            
            labels_list.append({"input_ids": label})
            dec_input_features.append({"input_ids": dec_input_feature})
        
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(labels_list, return_tensors="pt")
        dec_input_batch = self.processor.tokenizer.pad(dec_input_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels

        dec_input_features = dec_input_batch["input_ids"]
        if (dec_input_features[:, 0] == self.decoder_start_token_id).all().cpu().item():
            dec_input_features = dec_input_features[:, 1:]
        batch["dec_input_features"] = dec_input_features

        return batch

# Example usage
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
    apply_augmentation=True  # Set to False to disable augmentation
)


In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int
    apply_augmentation: bool = False
    n_fft_choices: List[int] = (400, 800, 1024)
    hop_length_choices: List[int] = (160, 320, 512)
    apply_noise_injection: bool = False  # Toggle for noise injection
    noise_profiles: List[str] = ('white', 'pink', 'environmental')  # Example noise profiles

    def add_noise(self, audio, noise_type='white', intensity=0.005):
        noise = torch.randn_like(audio) * intensity
        if noise_type == 'pink':
            noise = torchaudio.functional.highpass_biquad(noise, sample_rate=16000, cutoff_freq=200)
        elif noise_type == 'environmental':
            # Load an example environmental noise file
            noise, _ = torchaudio.load('environmental_noise.wav')
            noise = torch.nn.functional.interpolate(noise.unsqueeze(0), size=audio.size()).squeeze() * intensity
        return audio + noise

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = []
        labels_list = []
        dec_input_features = []
        
        for feature in features:
            audio = feature["input_features"]
            if self.apply_augmentation:
                n_fft = random.choice(self.n_fft_choices)
                hop_length = random.choice(self.hop_length_choices)
                if self.apply_noise_injection:
                    noise_type = random.choice(self.noise_profiles)
                    audio = self.add_noise(audio, noise_type=noise_type)
            else:
                n_fft = 1024
                hop_length = 512

            mel_spectrogram = torchaudio.transforms.MelSpectrogram(
                sample_rate=16000,  # Sample rate is assumed; update if necessary
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=80
            )(torch.tensor(audio))

            log_mel_spectrogram = torch.log(mel_spectrogram + 1e-9)
            input_features.append({"input_features": log_mel_spectrogram})
            
            label = feature["labels"]
            label_tokens = [self.processor.tokenizer.bos_token_id] + self.processor.tokenizer.encode(label) + [self.processor.tokenizer.eos_token_id]
            dec_input_feature = label_tokens[:-1]
            label = label_tokens[1:]
            
            labels_list.append({"input_ids": label})
            dec_input_features.append({"input_ids": dec_input_feature})
        
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(labels_list, return_tensors="pt")
        dec_input_batch = self.processor.tokenizer.pad(dec_input_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels

        dec_input_features = dec_input_batch["input_ids"]
        if (dec_input_features[:, 0] == self.decoder_start_token_id).all().cpu().item():
            dec_input_features = dec_input_features[:, 1:]
        batch["dec_input_features"] = dec_input_features

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
    apply_augmentation=True,  # Set to True to enable augmentation
    apply_noise_injection=True  # Set to True to enable noise injection
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=data_collator)

for batch in dataloader:
    # Pass the batch to your model
    outputs = model(batch)


In [ ]:
import torch
import torchaudio
import random
import numpy as np

def add_noise(audio, noise_type='white', intensity=0.005):
    noise = torch.randn_like(audio) * intensity
    if noise_type == 'pink':
        noise = torchaudio.functional.highpass_biquad(noise, sample_rate=16000, cutoff_freq=200)
    elif noise_type == 'environmental':
        # Load an example environmental noise file
        noise, _ = torchaudio.load('environmental_noise.wav')
        noise = torch.nn.functional.interpolate(noise.unsqueeze(0), size=audio.size()).squeeze() * intensity
    return audio + noise

def collate_fn(batch, apply_augmentation_flag=True, apply_noise_injection_flag=False):
    n_fft_choices = [400, 800, 1024]
    hop_length_choices = [160, 320, 512]
    noise_profiles = ['white', 'pink', 'environmental']

    input_features, labels, dec_input_features = [], [], []
    
    for f in batch:
        # Convert audio to features here
        audio = whisper.pad_or_trim(f["audio"].flatten())
        
        if apply_augmentation_flag:
            n_fft = random.choice(n_fft_choices)
            hop_length = random.choice(hop_length_choices)
            if apply_noise_injection_flag:
                noise_type = random.choice(noise_profiles)
                audio = add_noise(audio, noise_type=noise_type)
        else:
            n_fft = 1024
            hop_length = 512

        # Apply MelSpectrogram transformation with the selected parameters
        mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,  # Assuming a sample rate of 16000
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=80
        )(audio)

        # Apply logarithm for log-Mel spectrogram
        input_feature = torch.log(mel_spectrogram + 1e-9)

        label = f["label"]
        label_tokens = [tokenizer.bos_token_id] + tokenizer.encode(label) + [tokenizer.eos_token_id]
        dec_input_feature = label_tokens[:-1]
        label = label_tokens[1:]

        input_features.append(input_feature)
        labels.append(label)
        dec_input_features.append(dec_input_feature)

    input_features = torch.stack(input_features)

    max_label_len = max(len(l) for l in labels)
    max_dec_input_len = max(len(d) for d in dec_input_features)
    max_len = max(max_label_len, max_dec_input_len)

    labels = [np.pad(l, (0, max_len - len(l)), 'constant', constant_values=-100) for l in labels]
    dec_input_features = [np.pad(d, (0, max_len - len(d)), 'constant', constant_values=tokenizer.pad_token_id) for d in dec_input_features]

    # Convert the lists of numpy arrays to numpy arrays before creating tensors
    labels = np.array(labels)
    dec_input_features = np.array(dec_input_features)

    labels = torch.tensor(labels, dtype=torch.long)
    dec_input_features = torch.tensor(dec_input_features, dtype=torch.long)

    batch = {
        "input_features": input_features,
        "labels": labels,
        "dec_input_features": dec_input_features
    }
    return batch


In [ ]:
# Flag to apply augmentation
apply_augmentation_flag = True
apply_noise_injection_flag = True

# Create dataset and dataloader with augmentation and noise injection based on the flags
collate_fn_with_flags = partial(collate_fn, apply_augmentation_flag=apply_augmentation_flag, apply_noise_injection_flag=apply_noise_injection_flag)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn_with_flags)

for batch in dataloader:
    # Pass the batch to your model
    outputs = model(batch)
